In [1]:
import pandas as pd
from matplotlib import pyplot
from epiweeks import Week
from datetime import date, datetime
from os.path import dirname, join, isfile, isdir

from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf, normalize_igbe
from bulletin.commom.utils import isvaliddate

from bulletin.commom.utils import Timer, auto_fit_columns

In [2]:
municipios = static.municipios[['ibge','municipio','populacao']].copy()
municipios['ibge'] = municipios['ibge'].apply(str)

regionais = static.regionais[['ibge','nm_macro','nu_reg','nm_reg']].copy()
regionais['ibge'] = regionais['ibge'].apply(str)
regionais = regionais.rename(columns={'nu_reg':'rs'})

In [3]:
casos = pd.read_excel('input/casoseobitos.xlsx','casos',converters={
        'ibge': normalize_igbe
    }
)

In [4]:
obitos = pd.read_excel('input/casoseobitos.xlsx','obitos',converters={
        'ibge': normalize_igbe
    }
)

In [5]:
casos = pd.merge(left=casos, right=regionais, how='left', on='ibge')
casos = pd.merge(left=casos, right=municipios, how='left', on='ibge')
casos['ano'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).year, axis=1)
casos['se'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).week, axis=1)

In [6]:
obitos = pd.merge(left=obitos, right=regionais, how='left', on='ibge')
obitos = pd.merge(left=obitos, right=municipios, how='left', on='ibge')
obitos['ano'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).year, axis=1)
obitos['se'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).week, axis=1)

In [7]:
casos = casos.groupby(['ano','se','nm_macro','rs','nm_reg','ibge','municipio','populacao'])[['dt_diag']].count().rename(columns={'dt_diag':'casos'})
casos.reset_index(inplace=True)
casos['coef_incidencia'] = casos.apply(lambda x: x['casos']/x['populacao']*100000, axis=1)

casos

,ano,se,nm_macro,rs,nm_reg,ibge,municipio,populacao,casos,coef_incidencia
0,2020,11,LESTE,2,METROPOLITANA,410690,Curitiba,1948626,10,0.513182
1,2020,11,NOROESTE,13,CIANORTE,410550,Cianorte,83816,1,1.193090
2,2020,11,NOROESTE,15,MARINGA,411520,Maringá,430157,1,0.232473
3,2020,12,LESTE,1,PARANAGUA,411820,Paranaguá,156174,1,0.640311
4,2020,12,LESTE,2,METROPOLITANA,410420,Campo Largo,133865,1,0.747021
...,...,...,...,...,...,...,...,...,...,...
13874,2021,8,OESTE,20,TOLEDO,411460,Marechal Cândido Rondon,53495,11,20.562669
13875,2021,8,OESTE,20,TOLEDO,412085,Quatro Pontes,4029,1,24.820055
13876,2021,8,OESTE,20,TOLEDO,412740,Terra Roxa,17522,2,11.414222
13877,2021,8,OESTE,20,TOLEDO,412770,Toledo,142645,1,0.701041


In [8]:
obitos = obitos.groupby(['ano','se','nm_macro','rs','nm_reg','ibge','municipio','populacao'])[['data_obito']].count().rename(columns={'data_obito':'obitos'})
obitos.reset_index(inplace=True)
obitos['coef_mortalidade'] = obitos.apply(lambda x: x['obitos']/x['populacao']*100000, axis=1)

obitos

,ano,se,nm_macro,rs,nm_reg,ibge,municipio,populacao,obitos,coef_mortalidade
0,2020,13,NOROESTE,15,MARINGA,411520,Maringá,430157,3,0.697420
1,2020,14,LESTE,2,METROPOLITANA,410690,Curitiba,1948626,1,0.051318
2,2020,14,NOROESTE,11,CAMPO MOURAO,410430,Campo Mourão,95488,3,3.141756
3,2020,14,NOROESTE,13,CIANORTE,410550,Cianorte,83816,2,2.386179
4,2020,14,NORTE,17,LONDRINA,411370,Londrina,575377,1,0.173799
...,...,...,...,...,...,...,...,...,...,...
3491,2021,8,NORTE,19,JACAREZINHO,412850,Wenceslau Braz,19386,1,5.158362
3492,2021,8,OESTE,10,CASCAVEL,410305,Boa Vista da Aparecida,7540,1,13.262599
3493,2021,8,OESTE,10,CASCAVEL,410480,Cascavel,332333,2,0.601806
3494,2021,8,OESTE,10,CASCAVEL,412090,Quedas do Iguaçu,34409,1,2.906216


In [9]:
casos_obitos = pd.merge(casos,obitos,how='outer').fillna(0)
casos_obitos['coef_letalidade'] = casos_obitos.apply(lambda x: x['obitos']/x['casos'] if x['casos'] != 0 else 0, axis=1)
casos_obitos['casos'] = casos_obitos['casos'].apply(int)
casos_obitos['obitos'] = casos_obitos['obitos'].apply(int)
casos_obitos

,ano,se,nm_macro,rs,nm_reg,ibge,municipio,populacao,casos,coef_incidencia,obitos,coef_mortalidade,coef_letalidade
0,2020,11,LESTE,2,METROPOLITANA,410690,Curitiba,1948626,10,0.513182,0,0.000000,0.0
1,2020,11,NOROESTE,13,CIANORTE,410550,Cianorte,83816,1,1.193090,0,0.000000,0.0
2,2020,11,NOROESTE,15,MARINGA,411520,Maringá,430157,1,0.232473,0,0.000000,0.0
3,2020,12,LESTE,1,PARANAGUA,411820,Paranaguá,156174,1,0.640311,0,0.000000,0.0
4,2020,12,LESTE,2,METROPOLITANA,410420,Campo Largo,133865,1,0.747021,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13945,2021,4,OESTE,7,PATO BRANCO,412665,Sulina,2930,0,0.000000,1,34.129693,0.0
13946,2021,5,NOROESTE,14,PARANAVAI,411830,Paranapoema,3241,0,0.000000,1,30.854674,0.0
13947,2021,5,NORTE,17,LONDRINA,412033,Prado Ferreira,3780,0,0.000000,1,26.455026,0.0
13948,2021,7,NOROESTE,14,PARANAVAI,412420,Santo Antônio do Caiuá,2626,0,0.000000,1,38.080731,0.0


In [11]:
casos_obitos_regionais = casos_obitos.groupby(['ano','se','nm_macro','rs','nm_reg'])[['populacao','casos','obitos']].sum()

casos_obitos_regionais['coef_incidencia'] = casos_obitos_regionais.apply(lambda x: x['casos']/x['populacao']*100000, axis=1)
casos_obitos_regionais['coef_mortalidade'] = casos_obitos_regionais.apply(lambda x: x['obitos']/x['populacao']*100000, axis=1)
casos_obitos_regionais['coef_letalidade'] = casos_obitos_regionais.apply(lambda x: x['obitos']/x['casos'] if x['casos'] != 0 else 0, axis=1)
casos_obitos_regionais = casos_obitos_regionais.drop(columns=['populacao'])
casos_obitos_regionais

casos  obitos  coef_incidencia  \
ano  se nm_macro rs nm_reg                                              
2020 11 LESTE    2  METROPOLITANA         10       0         0.513182   
        NOROESTE 13 CIANORTE               1       0         1.193090   
                 15 MARINGA                1       0         0.232473   
     12 LESTE    1   PARANAGUA             1       0         0.640311   
                 2  METROPOLITANA         53       0         2.391717   
...                                      ...     ...              ...   
2021 8  OESTE    7  PATO BRANCO           74       0        37.664017   
                 8  FRANCISCO BELTRAO    120       0        38.463881   
                 9  FOZ DO IGUACU         49       0        18.172850   
                 10 CASCAVEL              44       4         8.434322   
                 20 TOLEDO                22       1         7.522653   

                                       coef_mortalidade  coef_letalidade  
ano  se nm_macro rs nm_reg                                                
2020 11 LESTE    2  METROPOLITANA              0.000000         0.000000  
        NOROESTE 13 CIANORTE                   0.000000         0.000000  
                 15 MARINGA                    0.000000         0.000000  
     12 LESTE    1   PARANAGUA                 0.000000         0.000000  
                 2  METROPOLITANA              0.000000         0.000000  
...                                                 ...              ...  
2021 8  OESTE    7  PATO BRANCO                0.000000         0.000000  
                 8  FRANCISCO BELTRAO          0.000000         0.000000  
                 9  FOZ DO IGUACU              0.000000         0.000000  
                 10 CASCAVEL                   0.766757         0.090909  
                 20 TOLEDO                     0.341939         0.045455  

[1070 rows x 5 columns]

In [13]:
writer = pd.ExcelWriter(join('output','coeficientes.xlsx'),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos_obitos.to_excel(writer,f"municipios")
worksheet = writer.sheets[f"municipios"]
auto_fit_columns(worksheet,casos_obitos)

casos_obitos_regionais.to_excel(writer,f"regionais")
worksheet = writer.sheets[f"regionais"]
auto_fit_columns(worksheet,casos_obitos_regionais)

writer.save()
writer.close()